In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from PIL import Image
import math
import os
import time

In [9]:
class CIFAR10Instance(torchvision.datasets.CIFAR10):
    """CIFAR10Instance Dataset."""
    def __getitem__(self, index):
        if self.train:
            img, target = self.data[index], self.targets[index]
        else:
            img, target = self.data[index], self.targets[index]

        img = Image.fromarray(img) # 使用 Image.fromarray(img) 将从数据集中获取的 NumPy 数组（代表图像数据）转换为 PIL.Image.Image 对象，以便后续的图像处理操作

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None: # 如果 self.target_transform 不为 None，则对标签应用相应的转换。这可以用于标签的转换，例如将标签从一个格式转换到另一个格式
            target = self.target_transform(target)

        return img, target, index

`target_transform` 通常用于对标签进行转换操作。例如，你可以使用它将分类标签从一个格式转换为另一个格式，或者对标签应用一些预处理操作。

以下是一个简单的示例，展示如何使用 `target_transform` 来将标签从整数形式转换为独热编码（one-hot encoding）形式：

### 示例：将标签转换为独热编码
```python
import torch
import torchvision
from torchvision import transforms
import numpy as np
from PIL import Image

# 定义一个将标签转换为独热编码的转换函数
def one_hot_encoding(target, num_classes=10):
    one_hot = np.zeros(num_classes, dtype=np.float32)
    one_hot[target] = 1.0
    return one_hot

# 扩展 CIFAR10 数据集并使用 target_transform 进行标签转换
class CIFAR10Instance(torchvision.datasets.CIFAR10):
    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]

        # 将 NumPy 数组转换为 PIL 图像
        img = Image.fromarray(img)

        # 应用图像转换
        if self.transform is not None:
            img = self.transform(img)

        # 应用标签转换
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, index

# 使用 target_transform 来将标签转换为独热编码
dataset = CIFAR10Instance(
    root='./data',
    train=True,
    download=True,
    transform=transforms.ToTensor(),
    target_transform=lambda target: one_hot_encoding(target, num_classes=10)
)

# 从数据集中获取一个样本
img, target, index = dataset[0]

print(f"Image Index: {index}")
print(f"Original Label (One-Hot Encoded): {target}")
```

### 解释：
1. **`one_hot_encoding` 函数**：该函数接收一个整数标签，将其转换为具有 `num_classes` 个元素的独热编码（one-hot encoding）数组。例如，如果标签为 `3`，那么输出将是 `[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]`。
2. **`target_transform` 参数**：当定义数据集时，使用 `target_transform=lambda target: one_hot_encoding(target, num_classes=10)` 将标签转换为独热编码。`lambda` 函数只是为了在定义数据集时传递参数到 `one_hot_encoding` 函数。
3. **`__getitem__` 方法**：在重写的 `__getitem__` 方法中，如果 `target_transform` 不为 `None`，则对标签应用 `target_transform` 转换。

### 输出：
- 输出的 `target` 是独热编码形式的标签，而不是单一的整数标签。
  
**注意**：独热编码在深度学习中并不总是必要的，因为大多数 PyTorch 损失函数（如 `CrossEntropyLoss`）能够直接处理整数形式的分类标签。不过，如果你需要在特定任务中手动处理标签，这种方式非常有用。

### 其他示例：
你可以使用 `target_transform` 执行其他类型的标签转换，例如将标签转换为浮点数、归一化标签值，或为多标签分类任务生成多个标签。

---

**a.** 是否需要进一步扩展 `target_transform` 进行更复杂的标签转换？  
**b.** 是否需要示例来处理回归任务中的标签转换？

In [10]:
import torch
import numpy as np

class AliasMethod(object):
    '''
        From: https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    '''
    def __init__(self, probs):

        if probs.sum() > 1:
            probs.div_(probs.sum())
            
        K = len(probs)
        self.prob = torch.zeros(K)
        self.alias = torch.LongTensor([0]*K)

        # Sort the data into the outcomes with probabilities
        # that are larger and smaller than 1/K.
        smaller = []
        larger = []
        for kk, prob in enumerate(probs):
            self.prob[kk] = K*prob
            if self.prob[kk] < 1.0:
                smaller.append(kk)
            else:
                larger.append(kk)

        # larger和smaller储存的是索引
        # Loop though and create little binary mixtures that
        # appropriately allocate the larger outcomes over the
        # overall uniform mixture.
        while len(smaller) > 0 and len(larger) > 0:
            small = smaller.pop()
            large = larger.pop()

            self.alias[small] = large
            self.prob[large] = (self.prob[large] - 1.0) + self.prob[small]

            if self.prob[large] < 1.0:
                smaller.append(large)
            else:
                larger.append(large)

        for last_one in smaller+larger:
            self.prob[last_one] = 1

    def cuda(self): 
        self.prob = self.prob.cuda()
        self.alias = self.alias.cuda()

    def draw(self, N):
        '''
            Draw N samples from multinomial
        '''
        K = self.alias.size(0)

        kk = torch.zeros(N, dtype=torch.long, device=self.prob.device).random_(0, K) # 生成全为0,长度为N的张量,再随机从0-(K-1)中抽取数填充
        prob = self.prob.index_select(0, kk)
        alias = self.alias.index_select(0, kk)
        # b is whether a random number is greater than q
        b = torch.bernoulli(prob)
        oq = kk.mul(b.long()) # mul 按元素相乘
        oj = alias.mul((1-b).long())

        return oq + oj

In [11]:
# import torch

# # 创建一个包含概率的张量
# prob = torch.tensor([0.2, 0.5, 0.8, 0.0, 1.0])

# # 使用 torch.bernoulli 生成伯努利随机数
# samples = torch.bernoulli(prob) 
# # 这个函数根据给定的概率值 prob，为每个元素生成一个随机数，如果该位置上的概率值大于或等于生成的随机数，则返回 1，否则返回 0。
# # 生成的 1 的概率与 prob 中对应的值相同，生成 0 的概率则为 1 - prob。
# print("Probability tensor:", prob)
# print("Bernoulli samples:", samples)

这段代码实现了 `Alias Method`，一种高效的多项分布采样方法，特别适用于具有大量离散结果的情况。它的作用是以 \(O(1)\) 的时间复杂度从给定的概率分布中采样。代码中定义了 `AliasMethod` 类，该类可以基于输入的概率分布生成样本。

### 逐行解释代码

```python
class AliasMethod(object):
    '''
        From: https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    '''
```
- 定义了一个 `AliasMethod` 类，该类用于实现 `Alias Method` 采样算法。注释中提供了参考资料的链接。

```python
def __init__(self, probs):
```
- 定义了类的构造函数 `__init__`，接收一个概率分布 `probs` 作为输入参数。

```python
if probs.sum() > 1:
    probs.div_(probs.sum())
```
- 如果输入的概率分布 `probs` 的和大于1，将 `probs` 归一化，使其和为1。`div_` 是一个原地操作（in-place operation），会直接修改 `probs`。

```python
K = len(probs)
self.prob = torch.zeros(K)
self.alias = torch.LongTensor([0]*K)
```
- 计算概率分布 `probs` 的长度 `K`，即离散事件的数量。初始化两个 `torch.Tensor` 对象：`self.prob` 用于存储归一化后的概率值，`self.alias` 用于存储“别名”索引，分别分配给 `K` 个事件。

```python
smaller = []
larger = []
```
- 初始化两个列表 `smaller` 和 `larger`，用于分别存储小于和大于 \( \frac{1}{K} \) 的概率的事件索引。

```python
for kk, prob in enumerate(probs):
    self.prob[kk] = K*prob
    if self.prob[kk] < 1.0:
        smaller.append(kk)
    else:
        larger.append(kk)
```
- 遍历 `probs`，将每个概率 `prob` 乘以 `K` 后存入 `self.prob` 中。如果处理后的概率小于 1.0，将对应索引加入 `smaller` 列表，否则加入 `larger` 列表。

```python
while len(smaller) > 0 and len(larger) > 0:
    small = smaller.pop()
    large = larger.pop()

    self.alias[small] = large
    self.prob[large] = (self.prob[large] - 1.0) + self.prob[small]

    if self.prob[large] < 1.0:
        smaller.append(large)
    else:
        larger.append(large)
```
- 在 `smaller` 和 `larger` 列表都不为空时，循环执行以下步骤：
  - 从 `smaller` 和 `larger` 中各取出一个索引 `small` 和 `large`。
  - 将 `large` 的索引存储到 `self.alias[small]` 中，即为 `small` 创建“别名”。
  - 调整 `large` 对应的概率值，使其减去 `1.0` 并加上 `small` 对应的概率值。
  - 根据调整后的概率值决定将 `large` 重新加入 `smaller` 或 `larger`。

```python
for last_one in smaller+larger:
    self.prob[last_one] = 1
```
- 将 `smaller` 和 `larger` 列表中剩下的索引对应的 `self.prob` 值设为 1.0。这是为了处理剩余的概率，确保每个事件都有合适的概率值。

```python
def cuda(self): 
    self.prob = self.prob.cuda()
    self.alias = self.alias.cuda()
```
- 定义了 `cuda` 方法，用于将 `self.prob` 和 `self.alias` 两个张量移动到 GPU 上，以加速计算。

```python
def draw(self, N):
    '''
        Draw N samples from multinomial
    '''
    K = self.alias.size(0)
```
- 定义 `draw` 方法，用于从多项分布中抽取 `N` 个样本。首先，获取 `self.alias` 的大小 `K`。

```python
kk = torch.zeros(N, dtype=torch.long, device=self.prob.device).random_(0, K)
prob = self.prob.index_select(0, kk)
alias = self.alias.index_select(0, kk)
```
- 生成 `N` 个随机索引 `kk`，这些索引指向 `self.prob` 和 `self.alias` 中的元素。`prob` 和 `alias` 分别选择 `kk` 索引指向的 `self.prob` 和 `self.alias` 中的值。

```python
b = torch.bernoulli(prob)
oq = kk.mul(b.long())
oj = alias.mul((1-b).long())
```
- 对每个 `prob` 生成一个伯努利随机数 `b`。然后根据 `b` 的值，确定使用原索引 `kk` 还是“别名”索引 `alias`。

```python
return oq + oj
```
- 返回最终的索引值，这些索引对应于从给定概率分布中采样得到的结果。

### 总结：
这段代码实现了 `Alias Method`，这是一个用于高效从离散分布中采样的算法。通过使用 `prob` 和 `alias` 表，可以在 \(O(1)\) 时间内生成样本。`cuda` 方法允许在 GPU 上执行操作，而 `draw` 方法用于实际进行采样。

**a.** 是否需要展示一个使用 `AliasMethod` 类生成样本的具体实例？  
**b.** 是否需要解释如何应用此类在特定任务中，如生成模型或其他概率分布采样任务？

In [12]:
# test
import torch

memory = torch.randn(100, 64)  # 假设 memory bank 有 100 个样本，每个样本的特征维度为 64
idx = torch.tensor([
    [10, 20, 30],  # 第一个样本：正样本索引为 10，负样本索引为 20 和 30
    [40, 50, 60],  # 第二个样本：正样本索引为 40，负样本索引为 50 和 60
    [70, 80, 90],  # 第三个样本：正样本索引为 70，负样本索引为 80 和 90
    [ 0,  1,  2]   # 第四个样本：正样本索引为  0，负样本索引为  1 和  2
])
weight = torch.index_select(memory, 0, idx.view(-1)) # 从memory中挑选指定的行
weight.shape

torch.Size([12, 64])

In [13]:
import torch
from torch.autograd import Function
from torch import nn
# from .alias_multinomial import AliasMethod
import math

class NCEFunction(Function):
    @staticmethod
    def forward(self, x, y, memory, idx, params): # features.shape=(batch_size, feature_size), indexes.shape=torch.Size([128]), memory bank, idx.shape(128,4097), params
        K = int(params[0].item())  # 负样本的数量
        T = params[1].item()  # 温度参数
        Z = params[2].item()  # 归一化常数
        momentum = params[3].item()  # 动量参数
        batchSize = x.size(0)  # 当前批次的大小
        outputSize = memory.size(0)  # memory bank 的大小
        inputSize = memory.size(1)  # 输入特征的维度

        # sample positives & negatives
        idx.select(1,0).copy_(y.data) # 将正样本索引放入 idx 的第一列，这样idx的第一列就都是正样本，后面4096个都是负样本

        # 采样相应的特征向量（正样本和负样本）
        weight = torch.index_select(memory, 0, idx.view(-1)) # 从memory中提取出所有第0维（行），再按照idx.view(-1)的索引提取出指定的行 - (len(idx.view(-1)), inputSize)
        # 感觉也不能说是weights吧，许多images的特征，用features似乎更合适一点
        weight.resize_(batchSize, K+1, inputSize) # resize成跟idx一样的shape - (batchSize, K+1, inputSize) - (128, 4097, 128)

        # inner product
        out = torch.bmm(weight, x.data.resize_(batchSize, inputSize, 1)) # x.shape=(128,128), 所以这里的out就是(128, 4097, 1)，每个正样本和4097个样本（一正4096副）的内积
        # 非参数softmax
        # print("1", out)
        out.div_(T).exp_() # batchSize * self.K+1
        # print("2", out)
        x.data.resize_(batchSize, inputSize)

        if Z < 0: # Z 的设置: 如果 Z 小于 0，表示还没有初始化，因此在第一次计算时设置 Z 的值 - Z就是非参数softmax的分母
            params[2] = out.mean() * outputSize # out.mean()生成单个数
            Z = params[2].item() 
            print("normalization constant Z is set to {:.1f}".format(Z))

        out.div_(Z).resize_(batchSize, K+1) # 从(batchSize, K+1, 1) -> (batchSize, K+1)

        # 保存用于反向传播的张量
        self.save_for_backward(x, memory, y, weight, out, params)

        return out

    @staticmethod
    def backward(self, gradOutput):
        x, memory, y, weight, out, params = self.saved_tensors
        K = int(params[0].item())
        T = params[1].item()
        Z = params[2].item()
        momentum = params[3].item()
        batchSize = gradOutput.size(0)
        
        # gradients d Pm / d linear = exp(linear) / Z????????? # 此时开始更新特征v，原文中equation2
        gradOutput.data.mul_(out.data) # out.shape=(batchSize, K+1) # 应该是有一个近似，具体看平板
        # add temperature # d exp(linear) / d (v_i)^T v = (1/T) * exp((v_i)^T v)
        gradOutput.data.div_(T)

        gradOutput.data.resize_(batchSize, 1, K+1)
        
        # gradient of linear 
        # print(gradOutput.shape, weight.shape) # torch.Size([128, 4097]) torch.Size([128, 4097, 128])
        gradOutput = gradOutput.reshape(batchSize, 1, K+1) # 这一步源代码中没有
        # d exp((v_i)^T v) / d v_i = exp((v_i)^T v) * v 吗？？？？？？？？？
        # \exp((v_i)^T v) \) 对 \( v_i \) 的导数是 exp(v_i^T v) \cdot v - GPT
        # v就是weight - 果然有近似
        gradInput = torch.bmm(gradOutput.data, weight) # (batchSize, 1, K+1) mm (batchSize, K+1, inputSize) -> (batchSize, 1, inputSize)
        gradInput.resize_as_(x) # x.shape=(batch_size, feature_size=inputSize)

        # update the non-parametric data - 更新memory bank中的特征v
        # weight.shape =(batchSize, K+1, inputSize)
        weight_pos = weight.select(1, 0).resize_as_(x) # 见下面的test - (batchSize, inputSize) 及所有批次的第一个样本的特征，及所有正样本的特征
        weight_pos.mul_(momentum)
        weight_pos.add_(torch.mul(x.data, 1-momentum)) # v_i' = \text{momentum} \cdot v_i + (1 - \text{momentum}) \cdot x
        w_norm = weight_pos.pow(2).sum(1, keepdim=True).pow(0.5)
        updated_weight = weight_pos.div(w_norm) # 将更新后的向量归一化，确保它的模为 1
        memory.index_copy_(0, y, updated_weight)
        
        return gradInput, None, None, None, None

In [33]:
# test
import torch

batchSize = 4
K = 6
inputSize = 6

weight = torch.randn(2, 2, 2)  # 初始形状为 (2, 2, 2)
weight.resize_(batchSize, K+1, inputSize)  # 调整形状为 (4, 7, 6)
selected_weight = weight.select(1, 0)
weight.shape, selected_weight.shape

(torch.Size([4, 7, 6]), torch.Size([4, 6]))

对于给定的函数 $P(i|v) = \dfrac{\exp\left(\frac{v_i^T v}{t}\right)}{\sum_{j=1}^n \exp\left(\frac{v_j^T v}{t}\right)}$，我们对 \( v_i \) 求导。这里 \( t \) 是一个常数，称为温度参数。

我们可以将这个过程分为以下几个步骤：

### 步骤 1: 定义函数
首先，我们可以将 \( P(i|v) \) 写成两部分的比值：

$$
P(i|v) = \frac{A_i}{B}
$#

其中：

$$
A_i = \exp\left(\frac{v_i^T v}{t}\right)
$$
$$
B = \sum_{j=1}^n \exp\left(\frac{v_j^T v}{t}\right)
$$

### 步骤 2: 对 \( A_i \) 求导
对 \( A_i \) 关于 \( v_i \) 求导：

$$
\frac{\partial A_i}{\partial v_i} = \frac{1}{t} \exp\left(\frac{v_i^T v}{t}\right) \cdot v = \frac{1}{t} A_i \cdot v
$$

### 步骤 3: 对 \( B \) 求导
对 \( B \) 关于 \( v_i \) 求导：

$$
\frac{\partial B}{\partial v_i} = \frac{\partial}{\partial v_i} \left(\sum_{j=1}^n \exp\left(\frac{v_j^T v}{t}\right)\right)
$$

注意到 \( B \) 是 \( n \) 项之和，其中只有第 \( i \) 项包含 \( v_i \)。因此，导数为：

$$
\frac{\partial B}{\partial v_i} = \frac{1}{t} \exp\left(\frac{v_i^T v}{t}\right) \cdot v = \frac{1}{t} A_i \cdot v
$$

### 步骤 4: 使用商的求导法则
使用商的求导法则（Quotient Rule），对 \( P(i|v) = \frac{A_i}{B} \) 关于 \( v_i \) 求导：

$$
\frac{\partial P(i|v)}{\partial v_i} = \frac{\partial \left(\frac{A_i}{B}\right)}{\partial v_i} = \frac{\frac{\partial A_i}{\partial v_i} \cdot B - A_i \cdot \frac{\partial B}{\partial v_i}}{B^2}
$$

将前面计算的 \( \frac{\partial A_i}{\partial v_i} \) 和 \( \frac{\partial B}{\partial v_i} \) 代入：

$$
\frac{\partial P(i|v)}{\partial v_i} = \frac{\left(\frac{1}{t} A_i \cdot v\right) \cdot B - A_i \cdot \left(\frac{1}{t} A_i \cdot v\right)}{B^2}
$$

### 步骤 5: 化简表达式
进一步化简这个表达式：

$$
\frac{\partial P(i|v)}{\partial v_i} = \frac{\frac{1}{t} A_i \cdot v \cdot B - \frac{1}{t} A_i^2 \cdot v}{B^2} = \frac{A_i \cdot v}{t} \cdot \frac{B - A_i}{B^2}
$$

等价于

$$
\frac{\partial P(i|v)}{\partial v_i} = \frac{\frac{1}{t} A_i \cdot v \cdot B - \frac{1}{t} A_i^2 \cdot v}{B^2} = \frac{A_i}{B} \cdot \frac{1}{t} \cdot v \cdot \frac{B - A_i}{B}
$$

注意到 \( P(i|v) = \frac{A_i}{B} \)，所以可以将表达式重新整理为：

$$
\frac{\partial P(i|v)}{\partial v_i} = P(i|v) \cdot \frac{v}{t} \cdot \left(1 - P(i|v)\right)
$$

### 最终结果
所以，对 \( P(i|v) \) 关于 \( v_i \) 的导数为：

$$
\frac{\partial P(i|v)}{\partial v_i} = P(i|v) \cdot \frac{v}{t} \cdot \left(1 - P(i|v)\right)
$$

In [14]:
class NCEAverage(nn.Module):

    def __init__(self, inputSize, outputSize, K, T=0.07, momentum=0.5, Z=None):
        super(NCEAverage, self).__init__()
        self.nLem = outputSize
        self.unigrams = torch.ones(self.nLem)
        self.multinomial = AliasMethod(self.unigrams)
        self.multinomial.cuda()
        self.K = K

        self.register_buffer('params',torch.tensor([K, T, -1, momentum]));
        stdv = 1. / math.sqrt(inputSize/3)
        self.register_buffer('memory', torch.rand(outputSize, inputSize).mul_(2*stdv).add_(-stdv))
 
    def forward(self, x, y): # lemniscate(features, indexes); x - (batch_size, feature_size); y - (batch_size)
        batchSize = x.size(0)
        idx = self.multinomial.draw(batchSize * (self.K+1)).view(batchSize, -1)
        # print("!!!", idx.shape, idx) # torch.Size([128, 4097])
        out = NCEFunction.apply(x, y, self.memory, idx, self.params)
        return out

In [15]:
class NCECriterion(nn.Module):
    def __init__(self, nLem):
        super(NCECriterion, self).__init__()
        self.nLem = nLem

    def forward(self, x, targets):
        batchSize = x.size(0)
        K = x.size(1) - 1
        Pnt = 1 / float(self.nLem)
        Pns = 1 / float(self.nLem)

        Pmt = x.select(1, 0)
        Pmt_div = Pmt.add(K * Pnt + 1e-7)
        lnPmt = torch.div(Pmt, Pmt_div)

        Pon_div = x.narrow(1, 1, K).add(K * Pns + 1e-7)
        # print("###########################", Pon_div)
        Pon = Pon_div.clone().fill_(K * Pns)
        lnPon = torch.div(Pon, Pon_div)

        lnPmt.log_()
        lnPon.log_()

        lnPmtsum = lnPmt.sum(0)
        lnPonsum = lnPon.view(-1, 1).sum(0)

        ######################################################
        print("###", lnPmtsum.item(), lnPonsum.item(), "###",)
        loss = - (lnPmtsum + lnPonsum) / batchSize
        return loss

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0
start_epoch = 0
low_dim = 128
nce_k = 4096 # defult 4096
nce_t = 0.5 # 温度
nce_m = 0.5 # SGD 动量参数
learning_rate = 0.0001
batch_size = 128
num_workers = 0
num_epochs = 200

In [17]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(size=32, scale=(0.2, 1.)),
    # transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# d2l-zh/pytorch/MyExercises/data
trainset = CIFAR10Instance(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

testset = CIFAR10Instance(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=num_workers)

ndata = len(trainset)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [18]:
for img, target, index in trainloader:
    print(img.shape, target.shape, index.shape)
    print(target, index)
    break

torch.Size([128, 3, 32, 32]) torch.Size([128]) torch.Size([128])
tensor([3, 0, 6, 2, 3, 5, 7, 5, 7, 3, 8, 6, 1, 6, 8, 0, 2, 8, 7, 7, 2, 9, 9, 8,
        5, 8, 0, 2, 9, 7, 6, 3, 5, 1, 6, 1, 2, 9, 8, 5, 6, 1, 7, 7, 7, 3, 3, 0,
        6, 0, 7, 9, 6, 5, 4, 0, 4, 3, 6, 8, 8, 2, 6, 2, 0, 0, 9, 1, 9, 1, 6, 6,
        3, 4, 1, 4, 1, 0, 1, 1, 9, 5, 6, 5, 1, 7, 2, 3, 3, 0, 6, 5, 2, 7, 8, 1,
        1, 1, 2, 1, 7, 5, 2, 2, 2, 6, 9, 4, 7, 4, 8, 5, 3, 3, 9, 1, 5, 1, 3, 8,
        0, 4, 0, 5, 5, 5, 5, 1]) tensor([47683, 35798, 42358, 27738, 38423, 19606, 21718,  3896,  2977, 28330,
        31354, 42762, 17726,   488, 37794, 44666,  3558, 49985, 39962, 15045,
        30470, 34502, 47944,   328, 42317, 43846, 15992, 17859, 22160, 40199,
        43977,  6167, 35609, 24540, 20548, 29264, 48364, 34474, 38495, 26221,
        44862, 47532, 18524, 25788,  3782, 38100,  8302, 39372, 13046, 47862,
        10887, 13529, 49459, 12322,  5950, 41762, 11927,  1534, 37825, 45937,
         7447, 42430, 35349, 22835

In [19]:
print('==> Building model..')
net = torchvision.models.resnet18(num_classes=low_dim)  # Using ResNet18 with the low_dim output
lemniscate = NCEAverage(low_dim, ndata, nce_k, nce_t, nce_m)

# if device == 'cuda':
#     net = nn.DataParallel(net).to(device)
#     cudnn.benchmark = True
net.to(device)
lemniscate.to(device)

criterion = NCECriterion(ndata).to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
# optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=5e-4)

==> Building model..


In [20]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 80 epochs"""
    lr = learning_rate
    if epoch >= 80:
        lr *= 0.1 ** ((epoch - 80) // 40)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [21]:
def train(epoch):
    print(f'\nEpoch: {epoch}')
    adjust_learning_rate(optimizer, epoch)
    net.train()
    
    train_loss = 0
    for batch_idx, (inputs, targets, indexes) in enumerate(trainloader):
        inputs, targets, indexes = inputs.to(device), targets.to(device), indexes.to(device)
        optimizer.zero_grad()
        
        features = net(inputs)
        outputs = lemniscate(features, indexes)
        # print("###", outputs)
        loss = criterion(outputs, indexes)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        # if batch_idx % 100 == 0:
        #     print(f'Batch {batch_idx}/{len(trainloader)}: Loss: {loss.item():.4f}')
        print(f'Batch {batch_idx}/{len(trainloader)}: Loss: {loss.item():.4f}')

In [22]:
def test(epoch):
    global best_acc
    net.eval()
    acc = kNN(epoch, net, lemniscate, trainloader, testloader, 200, nce_t, 0)
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'lemniscate': lemniscate,
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

    print(f'Best Accuracy: {best_acc * 100:.2f}%')

In [23]:
for epoch in range(start_epoch, start_epoch + num_epochs):
    train(epoch)
    test(epoch)


Epoch: 0
!!! torch.Size([128, 4097]) tensor([[12761, 11460, 37707,  ..., 37701, 40144, 23955],
        [ 2290, 38044, 42255,  ...,   867, 18125, 40281],
        [ 3224,  8357, 39445,  ..., 22418, 14691, 19997],
        ...,
        [12396,  1240,  2812,  ..., 41168, 36731,  6092],
        [28842, 24197, 28147,  ...,   856, 48454, 36326],
        [20814, 20352, 22697,  ...,  8676, 32199, 28002]], device='cuda:0')
normalization constant Z is set to 468620.3
### -20.558202743530273 -1399154.625 ###


D:\Miniconda\envs\DL_learning\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Batch 0/391: Loss: 10931.0557
!!! torch.Size([128, 4097]) tensor([[38224,  5225,  9649,  ..., 34618, 36665, 36075],
        [14722, 12921, 11367,  ..., 47293, 32154, 13240],
        [  737, 19829, 27042,  ..., 23334, 26988, 40643],
        ...,
        [28256, 48599, 48520,  ..., 20615, 40464, 30319],
        [18862, 16471, 37304,  ..., 31582, 26305,  6677],
        [47730, 39393, 14373,  ...,  1060,  3899,  6174]], device='cuda:0')
### -19.122356414794922 -1386965.5 ###
Batch 1/391: Loss: 10835.8174
!!! torch.Size([128, 4097]) tensor([[10826,  7770, 30710,  ..., 25617, 34360, 40258],
        [27569,  7970, 38160,  ...,  6285, 15770, 36488],
        [26563, 32438, 39972,  ..., 48145,  2610, 18192],
        ...,
        [21042, 25809,  1394,  ...,  9848, 31683, 20836],
        [37446, 18136, 26267,  ...,  4381,   893, 32090],
        [29987,  9836,  3664,  ..., 10781, 22997, 33439]], device='cuda:0')
### -19.568439483642578 -1372033.75 ###
Batch 2/391: Loss: 10719.1670
!!! torch.Size([1


KeyboardInterrupt



In [ ]:
acc = kNN(0, net, lemniscate, trainloader, testloader, 200, nce_t, 1)
print(f'Final Accuracy: {acc * 100:.2f}%')